## Introdução às Seleções

Vamos começar com uma seleção básica: simplesmente clicar em algum ponto para realçá-lo. Usando o _dataset_ `cars`(carros), podemos iniciar com um _scatter plot_ de horsepower (cavalos de potência) x miles per gallon (milhas por galão), com uma codificação de cor para o número de cilindradas no carro por motor.

Além disso, iremos criar uma instância de seleção definida sobre um _único valor_. Por definição, a seleção usa o click do mouse para determinar o valor de seleção. Para registrar uma seleção com um gráfico, teremos que adicioná-la usando o método `.add_selection()`.

Uma vez que nossa seleção esteja bem definida, a usaremos como parâmetro para _codificações condicionais_, que aplicam uma codificação diferente caso o registo de dados esteja dentro ou fora da seleção. Por exemplo, considere o código a seguir:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey')).legend(title='Cilindradas')
~~~
Esta definição de codificação indica que os pontos de dados contidos em `selection` devem ser coloridos de acordo com o `Cylinder` (cilindradas) field, enquanto dados não selecionados devem usar um `grey` (cinza) por defeito. Uma seleção vazia inclui _todos_ os pontos de dados, e assim inicialmente todos os pontos serão coloridos.

_Tente clicar em diferentes pontos no gráfico abaixo. O que acontece?  (Clique no fundo para limpar o estado da seleção e voltar a uma seleção “vazia”.)_


In [ ]:
selection = alt.selection_single();

alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    alt.X('Horsepower:Q', title = 'Cavalos de Potência'),
    alt.Y('Miles_per_Gallon:Q', title = 'Milhas por Galão')
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')).legend(title='Cilindradas'),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

alt.Chart(...)

Claro, destacar pontos de dados individuais, um de cada vez, não é particularmente interessante! No entanto, como veremos, as seleções de valor único fornecem um bloco de construção útil para interações mais poderosas. Além disso, as seleções de valor único são apenas um dos três tipos de seleção fornecidos pelo Altair:

- `selection_single` - seleciona um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - seleciona múltiplos valores discretos. O primeiro valor é selecionado com o clique do mouse e os valores adicionais são alternados usando shift-click.
- `selection_interval` - seleciona um intervalo contínuo de valores, iniciado pelo arrastar do mouse.

Vamos comparar cada um destes tipos de seleção lado a lado. Para manter nosso código organizado, primeiro definiremos uma função (`plot`) que gera uma especificação de gráfico de dispersão como a acima. Podemos passar uma seleção para a função `plot` para que ela seja aplicada ao gráfico:


In [ ]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
    alt.X('Horsepower:Q', title = 'Cavalos de Potência'),
    alt.Y('Miles_per_Gallon:Q', title = 'Milhas por Galão')
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')).legend(title='Cilindradas'),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
).properties(
        width=240,
        height=180
    )

Vamos utilizar a nossa função `plot` para criar três variantes de gráfico, uma por tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta interações shift-click para alternar a inclusão de múltiplos pontos dentro da seleção. O terceiro gráfico (`interval`) gera uma região de seleção (ou _área de seleção_) ao arrastar o mouse. Uma vez criado, é possível arrastar a área de seleção para pontos diferentes ou rolar quando o cursor estiver dentro dessa área para dimensionar (zoom) seu tamanho.

Experimente interagir com cada um dos gráficos abaixo!_


In [ ]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Arraste)')
).encode(
    color=alt.Color('Cylinders:O', legend=alt.Legend(title='Cilindradas'))
)


alt.HConcatChart(...)

Os exemplos acima utilizam interações predefinidas (clicar, clicar com a tecla shift, arrastar) para cada tipo de seleção. Podemos personalizar ainda mais as interações fornecendo especificações de eventos de entrada usando a [sintaxe do seletor de eventos Vega] (https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para serem acionados em eventos `mouseover` (passar o mouse) em vez de eventos `click` (clique).

_Mantenha pressionada a tecla shift no segundo gráfico para “pintar” com os dados!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
).encode(
    color=alt.Color('Cylinders:O', legend=alt.Legend(title='Cilindradas'))
)

alt.HConcatChart(...)

Agora que cobrimos o básico das seleções com Altair, vamos fazer uma visita guiada às várias técnicas de interação possíveis!
